In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hide_1 = nn.Linear(784,500)
        self.hide_2 = nn.Linear(500,500)
        self.out = nn.Linear(500,10)

    def forward(self, x):
        x = F.relu(self.hide_1(x))
        x = F.relu(self.hide_2(x))
        x = self.out(x)
        return F.log_softmax(x, dim=1)


def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        num_img=data.size(0)
        # 将图片展开为28*28=784
        # view()函数作用是将一个多行的Tensor,拼接成一行,第一个参数是要拼接的tensor,第二个参数是-1
        data = data.view(num_img, -1)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    data_r=[]
    label_r=[]
    pre_r=[]
    with torch.no_grad():
        for data, target in test_loader:
            num_img=data.size(0)
            # view()函数作用是将一个多行的Tensor,拼接成一行
            # 第一个参数是要拼接的tensor,第二个参数是-1
            data_l = data.view(num_img, -1)  # 将图片展开为28*28=784
            data_l, target = data_l.to(device), target.to(device)
            output = model(data_l)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            temp=pred.eq(target.view_as(pred))
            correct += temp.sum().item()
            temp=temp.int()
            index=temp.argmin(0).item()
            data_r.append(data[index])
            label_r.append(target[index])
            pre_r.append(pred[index].item())

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return data_r,label_r,pre_r

In [ ]:
batch_size=64
test_batch_size=1000
epochs=2
lr=0.01
momentum=0.5
save_model=False
log_interval=10


device = torch.device("cpu")
kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

#展示数据集

plt.rcParams['font.sans-serif']=['SimHei']
num_pictures = 16
fig = plt.figure(figsize=(20,20))
current_fig = 1
for image, label in train_loader:
    if current_fig>num_pictures:
        break
    plt.subplot(4, 4, current_fig)
    plt.title('label:{}'.format(label[current_fig]))
    plt.imshow(image[current_fig].numpy().squeeze(), cmap='Greys_r')
    current_fig+=1
    


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
data_r=[]
label_r=[]
pre_r=[]
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    d_r,l_r,p_r=test(model, device, test_loader)
    data_r.extend(d_r)
    label_r.extend(l_r)
    pre_r.extend(p_r)
plt.figure(figsize=(20,20))
for i in range(len(data_r)):
    plt.subplot(2, 10, i+1)
    plt.title('label:{}, pre:{}'.format(label_r[i],pre_r[i]))
    plt.imshow(data_r[i].numpy().squeeze(), cmap='Greys_r')
